# System Analysis

# Initialization

In [ ]:
# Helper function to calculate Euclidean distance
import pandas as pd
from numpy import log10, sqrt

def euclid(x, y, df, take_log=False):
    if take_log:
        return sqrt(sum((log10(df[x])-log10(df[y]))**2))
    else:
        return sqrt(sum((df[x]-df[y])**2))
def sema(vect, fig):
    #semantics=', '.join('{}={}'.format(x,y) for (x,y) in fig.pepperargs['default'].items() if x != 'conc')
    nv = []
    for line in vect:
        v = []
        for (x,y) in fig.pepperargs[line].items():
            if x == 'conc': continue
            if x == 'max_complex_size': continue
            if x == 'max_complex_count': continue
            if x == 'max_reaction_count': continue
            if x == 'condensed': 
                z = x if y is True else 'detailed'
            else:
                z = '{}={}'.format(x,y)
            v.append(z)
        nv.append(', '.join(v))
    return nv

In [ ]:
from zhang2007 import data; z07 = data()
from yin2008 import data; y08 = data()
from zhang2009 import data; z09 = data()
from zhang2010 import data; z10 = data()
from zhang2011 import data; z11 = data()
from kotani2017 import data; k17 = data()

%who list

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="darkgrid")

In [ ]:
# Main Figures:
Table1 = pd.DataFrame()

# Peppercorn estimates
## Zhang 2007

In [ ]:
# Generate Peppercorn estimates for every FigureData setup.
for fig in z07:
    print(fig.name)
    #display(fig.pepperargs['default'])
    fig.eval(verbose=0)
    for df in fig.get_dataframes():
        df['Name']=fig.name
        df['Semantics']=sema(df['Semantics'], fig)
        #display(df)
        Table1 = Table1.append(df)


In [ ]:
tmpfig = Table1
tmpfig = tmpfig.loc[tmpfig['Name'].isin([fig.name for fig in z07])]
#display(tmpfig)
#g = sns.lmplot(x="Time (experiment)", y="Time (simulation)", hue='Name', data=tmpfig, fit_reg=False, markers=m, legend=True)
# Plot data
sns.relplot(x="Time (experiment)", y="Time (simulation)", col="Metric", hue='Name', data=tmpfig)
#plt.xscale('log')
#plt.yscale('log')
(mi, ma)=(1e0, 8e3)
#plt.xlim(mi, ma)
#plt.ylim(mi, ma)
plt.plot([mi, ma], [mi, ma], linewidth=1, color='white',zorder=0)

## Yin2008

In [ ]:
# Generate Peppercorn estimates for every FigureData setup.
for fig in y08:
    print(fig.name)
    display(fig.pepperargs['default'])
    fig.eval(verbose = 0)
    for df in fig.get_dataframes():
        df['Name']=fig.name
        df['Semantics']=sema(df['Semantics'], fig)
        #display(df)
        Table1 = Table1.append(df)

In [ ]:
tmpfig = Table1
tmpfig = tmpfig.loc[tmpfig['Name'].isin([fig.name for fig in y08])]
display(tmpfig)

sns.relplot(x="Time (experiment)", y="Time (simulation)", col = 'Metric', hue='Name', data=tmpfig)
#plt.xscale('log')
#plt.yscale('log')
(mi, ma)=(1e0, 1.2e4)
#plt.xlim(mi, ma)
#plt.ylim(mi, ma)
plt.plot([mi, ma], [mi, ma], linewidth=1, color='white',zorder=0)

## Zhang 2009 

In [ ]:
for fig in z09:
    print(fig.name)
    print(fig.pepperargs['default'])
    fig.eval(cmpfig='hack')
    for df in fig.get_dataframes():
        df['Name']=fig.name
        df['Semantics']=sema(df['Semantics'], fig)
        dist = euclid('Time (experiment)', 'Time (simulation)', df, take_log=True)
        print('{:s} -- Fitness: {:g}'.format(fig.name, dist)) 
        #display(df)
        Table1 = Table1.append(df)
        

In [ ]:
tmpfig = Table1
tmpfig = tmpfig.loc[tmpfig['Name'].isin([fig.name for fig in z09])]
display(tmpfig)

sns.relplot(x="Time (experiment)", y="Time (simulation)", col = 'Metric', hue='Name', data=tmpfig)
#plt.xscale('log')
#plt.yscale('log')
(mi, ma)=(1e0, 2e4)
#plt.xlim(mi, ma)
#plt.ylim(mi, ma)
plt.plot([mi, ma], [mi, ma], linewidth=1, color='white',zorder=0)

## Zhang 2010

In [ ]:
# Generate Peppercorn estimates for every FigureData setup.
for fig in z10:
    print(fig.name)
    #display(fig.pepperargs['default'])
    fig.eval(verbose = 1)
    for df in fig.get_dataframes():
        df['Name']=fig.name
        df['Semantics']=sema(df['Semantics'], fig)
        dist = euclid('Time (experiment)', 'Time (simulation)', df, take_log=True)
        print('{:s} -- Fitness: {:g}'.format(fig.name, dist)) 
        #display(df)
        Table1 = Table1.append(df)

In [ ]:
tmpfig = Table1
tmpfig = tmpfig.loc[tmpfig['Name'].isin([fig.name for fig in z10])]
display(tmpfig)
sns.relplot(x="Time (experiment)", y="Time (simulation)", col = 'Metric', hue='Name', data=tmpfig)
#plt.xscale('log')
#plt.yscale('log')
(mi, ma)=(1e0, 4e4)
#plt.xlim(mi, ma)
#plt.ylim(mi, ma)
plt.plot([mi, ma], [mi, ma], linewidth=1, color='white',zorder=0)

# Zhang 2011

In [ ]:
for fig in z11:
    print(fig.name)
    #print(fig.pepperargs['default'])
    fig.eval(verbose=1)
    for df in fig.get_dataframes():
        df['Name']=fig.name
        df['Semantics']=sema(df['Semantics'], fig)
        dist = euclid('Time (experiment)', 'Time (simulation)', df, take_log=True)
        print('{:s} -- Fitness: {:g}'.format(fig.name, dist)) 
        #display(df)
        Table1 = Table1.append(df)

In [ ]:
tmpfig = Table1
tmpfig = tmpfig.loc[tmpfig['Name'].isin([fig.name for fig in z11])]
#display(tmpfig)

sns.relplot(x="Time (experiment)", y="Time (simulation)", col='Metric', hue='Name', data=tmpfig)

#plt.xscale('log')
#plt.yscale('log')
(mi, ma)=(200, 1000)
#plt.xlim(mi, ma)
#plt.ylim(mi, ma)
plt.plot([mi, ma], [mi, ma], linewidth=1, color='white',zorder=0)

## Kotani & Hughes 2017

In [ ]:
# Generate Peppercorn estimates for every FigureData setup.
for fig in k17:
    print(fig.name)
    #display(fig.pepperargs['default'])
    fig.eval(verbose = 1)
    for df in fig.get_dataframes():
        df['Name']=fig.name
        df['Semantics']=sema(df['Semantics'], fig)
        dist = euclid('Time (experiment)', 'Time (simulation)', df, take_log=True)
        print('{:s} -- Fitness: {:g}'.format(fig.name, dist)) 
        #display(df)
        Table1 = Table1.append(df)

In [ ]:
tmpfig = Table1
tmpfig = tmpfig.loc[tmpfig['Name'].isin([fig.name for fig in k17])]
display(tmpfig)

sns.relplot(x="Time (experiment)", y="Time (simulation)", hue='Name', col='Metric', data=tmpfig)
#plt.xscale('log')
#plt.yscale('log')
(mi, ma)=(1e1, 1e5)
#plt.xlim(mi, ma)
#plt.ylim(mi, ma)
plt.plot([mi, ma], [mi, ma], linewidth=1, color='white',zorder=0)

# Everything

In [ ]:
#display(Table1)
Table1.to_latex('ManySystems.tex', index=False, float_format='{:.3g}'.format)
tmpfig = Table1
#tmpfig.reset_index(inplace=True, drop=True)
#tmpfig = tmpfig.iloc[0:-1]
#tmpfig = tmpfig.iloc[[4,5,6,7,28,29,30,31], :]
#display(tmpfig)

#sns.set_palette("husl")
#sns.set(rc={'figure.figsize':(4,4)})

m = ['+'] * 3
m += ['.']
m += ['^'] * 3
m += ['*'] * 4
m += ['v']
m += ['o'] * 3

#g = sns.relplot(x="Time (experiment)", y="Time (simulation)", hue='Name', data=tmpfig)
g = sns.lmplot(x = "Time (experiment)", y = "Time (simulation)", 
               col = 'Metric', hue = 'Name', sharey = True, col_order = ['completion-time', 'diagonal-crossing-time'],
               data = tmpfig, markers = m, 
               fit_reg = False, legend = False)

plt.xscale('log')
plt.yscale('log')
plt.legend(bbox_to_anchor=(1, -0.2), loc=1, borderaxespad=0.)#, fontsize = 10)
#plt.legend(loc='upper left')

#plt.xlim(1e3, 1e5)
#plt.ylim(1e2, 1e6)
mi, ma = 1e1, 1e5

for ax in g.axes[0]:
    ax.plot([mi, ma], [mi, ma], linewidth=3, color='white', zorder=0)

g.axes[0][0].set_title('50%-completion time')
g.axes[0][1].set_title('Diagonal-crossing time')

#g.set_titles(row_template="{row_name}", col_template="{col_name}")


g.set(xlabel='Experimental timepoint [s]')  
g.set(ylabel='Peppercorn timepoint [s]')

plt.savefig('all_systems.pdf', bbox_inches='tight')
plt.savefig('all_systems.svg', bbox_inches='tight')